In [1]:
!pip install ragstack-ai datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 759.9 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 941.9 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 3.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-llms-anthropic to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-llms-openai to determine which version is compatible with other requirements. This

In [6]:
import os
api_endpt = os.getenv("ASTRADB_API_ENDPOINT")
applcn_token = os.getenv("ASTRADB_APPLICATION_TOKEN")

In [11]:
from langchain_astradb import AstraDBVectorStore
from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings()
vector_store = AstraDBVectorStore(
    collection_name = "test",
    embedding = embedding,
    token = "AstraCS:ZhXukXhAgMPpfhAkceqOJCGH:f310832cf52102d9b91212dcb0ecad9ab37c7472a8d41631359ba767b1688ab8",
    api_endpoint = "https://bd040547-e365-4db6-8ed5-f94a41c3cafd-us-east-2.apps.astra.datastax.com"
)

print("Astra vector store configured")

Astra vector store configured


In [13]:
from datasets import load_dataset

philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
print(philo_dataset[16])

Generating train split: 100%|██████████| 450/450 [00:00<00:00, 710.14 examples/s]

{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


In [14]:
from langchain.schema import Document

docs = []
for entry in philo_dataset:
    metadata = {"author":entry["author"]}
    if entry["tags"]:
        for tag in entry["tags"].split(";"):
            metadata[tag] = "y"
    
    doc = Document(page_content = entry["quote"],metadata = metadata)
    docs.append(doc)

In [15]:
docs[0]

Document(page_content="True happiness comes from gaining insight and growing into your best possible self. Otherwise all you're having is immediate gratification pleasure, which is fleeting and doesn't grow you as a person.", metadata={'author': 'aristotle', 'knowledge': 'y'})

In [16]:
inserted_ids = vector_store.add_documents(docs)

In [17]:
print(vector_store.astra_db.collection("test").find())

{'data': {'documents': [{'_id': 'dd05c4020640419e8d1b363f220fa241', 'content': 'Restlessness is the hallmark of existence.', 'metadata': {'author': 'schopenhauer'}}, {'_id': 'b5ee5bcec59d40a983b2748d31f67665', 'content': 'What makes us heroic?--Confronting simultaneously our supreme suffering and our supreme hope.', 'metadata': {'author': 'nietzsche'}}, {'_id': '9b90974d137e47e2b521ea8bd1c1f7b7', 'content': 'I have tried sedulously not to laugh at the acts of man, nor to lament them, nor to detest them, but to understand them.', 'metadata': {'author': 'spinoza', 'knowledge': 'y', 'education': 'y', 'ethics': 'y'}}, {'_id': 'c2b597c7d5db46508fadb627a1d2d25b', 'content': 'The dream unites the grossest contradictions, permits impossibilities, sets aside the knowledge that influences us by day, and exposes us as ethically and morally obtuse.', 'metadata': {'author': 'freud', 'ethics': 'y'}}, {'_id': '8a4298b8d6964e56ae79f9704c9e18e6', 'content': 'Love is pleasure accompanied by the idea of 

In [18]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOllama
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.llms import Ollama

retriever = vector_store.as_retriever(search_kwargs = {"k":3})

prompt_template = """
Answer the following question based only on the given context. If the question is out of context, tell them that you dont know the answer politely.
Context: {context}
Question: {question}
Your answer:
"""

prompt = ChatPromptTemplate.from_template(prompt_template)
model = Ollama(model = "qwen2:1.5b")

chain = (
    {"context":retriever,"question": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

In [23]:
chain.invoke("In the given context, what is the most important to allow the brain and provide me the tags?. Dont answer in a single word")

'Sleep.'